# CRNN for Axial Images

In [4]:
import tensorflow as tf
import pandas as pd
import seaborn
from matplotlib import pyplot as plt
import numpy as np
import os
from tensorflow import keras

In [5]:
import os

def get_image_files(folder_path):
    """Get a sorted list of image files from a given folder based on the number before the underscore."""
    image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
    image_files.sort(key=lambda x: int(x.split('_')[0]))  # Sort based on the number before the underscore
    return image_files

def get_mri_image_paths(base_folder):
    """Create a 2D array where each index is a list of paths to JPEG images for each MRI image, sorted by the number before the underscore."""
    mri_image_paths = []
    for class_folder in os.listdir(base_folder):
        class_path = os.path.join(base_folder, class_folder)
        if os.path.isdir(class_path):
            for mri_folder in os.listdir(class_path):
                mri_path = os.path.join(class_path, mri_folder)
                if os.path.isdir(mri_path):
                    image_files = get_image_files(mri_path)
                    image_paths = [os.path.join(mri_path, img_file) for img_file in image_files]
                    mri_image_paths.append(image_paths)
    return mri_image_paths

# Define base folder
base_folder = 'Organized_Image_Data copy 2'

# Get MRI image paths
mri_image_paths = get_mri_image_paths(base_folder)

# mri_image_paths now contains a 2D array with paths to JPEG images sorted by the number before the underscore
# for i, paths in enumerate(mri_image_paths):
#     print(f"Image paths for MRI {i}: {paths}")

In [6]:
final_data = []
for x in mri_image_paths:
    start =  (len(x) // 2)  - 5
    lst = []
    for i in range(10):
        lst.append(x[start + i])
    final_data.append(lst)
        

In [7]:
#final_data

In [8]:
len(final_data)

434

In [9]:
ids = []
for lst in final_data:
    id_ = lst[i].split('_')
    id_num = id_[-1].strip(".jpg")
    ids.append(id_num)

In [ ]:

# Example list of IDs

# Load the CSV file into a pandas DataFrame
csv_file = "newAxial_6_26_2024.csv"  # Replace with your actual CSV file path
df = pd.read_csv(csv_file)

# Initialize lists for IDs and corresponding categories
y = []

# Find categories for each ID in id_list
for id_num in ids:
    row = df[df['Image Data ID'] == id_num]
    y.append(row['Group'].iloc[0])  # Assuming 'Category' is a single value per ID

# Print found IDs and categories
# print("y:", y)
# print(len(y))

# # Create train/test splits
# train_ids, test_ids, train_categories, test_categories = train_test_split(found_ids, categories, test_size=0.2, random_state=42)

# # Print train/test splits
# print("\nTrain IDs:", train_ids)
# print("Train Categories:", train_categories)
# print("\nTest IDs:", test_ids)
# print("Test Categories:", test_categories)


# BUILD CRNN

In [99]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Input, Dense, Flatten, Reshape, MaxPooling2D, BatchNormalization, ReLU, Dropout, Reshape, LSTM, TimeDistributed
from tensorflow.keras.regularizers import l2

In [100]:
shape_CNN = (10,256,256,32)
shape = (10,256,256,1)

In [127]:

def Model(shape):
    
    model = Sequential()
    
    # Add the first convolutional layer
    model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'), input_shape=shape))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    
    # Add the second convolutional layer
    model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    
    # Optionally, add a third convolutional layer
    model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu', padding='same')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    
    # Flatten the output and feed it into an LSTM layer
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(64))
    
    # Add a dropout layer
    model.add(Dropout(0.5))
    
    # Add a dense layer with softmax activation for classification
    model.add(Dense(3, activation='softmax'))  # Assuming 3 classes for Alzheimer's stages
    
    return model


In [128]:
model = Model(shape)

C:\Users\jetma\anaconda3\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [129]:
model.compile('adam',loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])

In [130]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed_54                  │ (None, 10, 256, 256, 32)    │             320 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_55                  │ (None, 10, 256, 256, 32)    │             128 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_56                  │ (None, 10, 128, 128, 32)    │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_57                  │ (None, 10, 128, 128, 64)    │          18,496 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_58                  │ (None, 10, 128, 128, 64)    │             256 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_59                  │ (None, 10, 64, 64, 64)      │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_60                  │ (None, 10, 64, 64, 128)     │          73,856 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_61                  │ (None, 10, 64, 64, 128)     │             512 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_62                  │ (None, 10, 32, 32, 128)     │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_63                  │ (None, 10, 131072)          │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ (None, 10, 64)              │      33,571,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 33,697,859 (128.55 MB)

 Trainable params: 33,697,411 (128.55 MB)

 Non-trainable params: 448 (1.75 KB)

In [105]:
# import tensorflow as tf
# import matplotlib.pyplot as plt
# from PIL import Image

# # Assuming train_dataset is your TensorFlow dataset
# for sequence, label in train_dataset.take(1):  # Take the first batch
#     plt.figure(figsize=(20, 2))  # Adjust the figure size as needed
    
#     for i, image_path in enumerate(sequence):
#         image_path_str = image_path.numpy().decode('utf-8')  # Convert bytes to string
#         image = Image.open(image_path_str)  # Load image using PIL (you can also use OpenCV)
        
#         plt.subplot(1, 10, i + 1)  # 1 row, 10 columns, i+1 is the index of the subplot
#         plt.imshow(image)
#         plt.title(f"Label: {label.numpy()}")
#         plt.axis("off")
    
#     plt.tight_layout()  # Adjust spacing between plots
#     plt.show()

In [106]:
# for sequence, label in train_dataset.take(1):  # Take the first batch
#     plt.figure(figsize=(20, 2))  # Adjust the figure size as needed
    
#     for i, image_path in enumerate(sequence):
#         image_path_str = image_path.numpy().decode('utf-8')  # Convert bytes to string
#         print(image_path_str)

In [107]:
# X_train

In [108]:
from PIL import Image
import numpy as np

def get_values(path):
    # Assuming final_data[0][0] contains the path to your image
    image_path = path
    
    # Open the image using PIL
    im = Image.open(image_path)
    
    # Convert PIL Image to numpy array
    im_array = np.array(im)
    
    # Now you can access and manipulate the image as a numpy array
    return(im_array)  # Check the shape of the numpy array


In [109]:
print(get_values(final_data[0][0]))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [110]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Example y labels (replace with your actual labels)

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit the encoder to the labels and transform them
y_numeric = label_encoder.fit_transform(y)

print("Original labels:", y)
print("Numeric labels:", y_numeric)


Original labels: ['AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN'

In [111]:
# import numpy as np
# from sklearn.model_selection import train_test_split
# import tensorflow as tf

# # Normalize the images
# X = np.array(total_images) / 255.0

# # Convert y to a numpy array if not already
# y_true = np.array(y_numeric)

# # Train/test split
# X_train_true, X_test_true, y_train_true, y_test_true = train_test_split(X, y_true, test_size=0.2, random_state=42)


In [112]:
total_images = []
counter = 10
for x in final_data:
    image_pixels = []
    for image in x:
        image_pixels.append(get_values(image))
    X_train_true = np.expand_dims(image_pixels, axis=-1)
    total_images.append(image_pixels)

In [113]:
total_images_test = np.array(total_images)

In [114]:
total_images_test.shape

(434, 10, 256, 256)

In [115]:
y_test = np.array(y_numeric)

In [116]:
y_test.shape

(434,)

In [117]:
total_images_test = total_images_test.reshape((434, 10, 256, 256, 1))

In [ ]:
history = model.fit(total_images_test, y_test, epochs=10)

Epoch 1/10
 1/14 ━━━━━━━━━━━━━━━━━━━━ 17:55 83s/step - accuracy: 0.2500 - loss: 1.1538